In [7]:
%reload_ext autoreload
%autoreload 2

In [8]:
import plumbum as pb
from vastai.api import VastClient
import time
from plumbum.machines.paramiko_machine import ParamikoMachine
from paramiko.client import AutoAddPolicy

In [9]:
vast = VastClient().authenticate()

api_key_file:  ~/.vast_api_key
Initializing vast.ai client with api_key from /home/sam/.vast_api_key.
Already logged in.


In [4]:
vast.get_running_instances()

[]

In [11]:
vast.get_instances()

[       ID Machine   Status  Num      Model Util. % vCPUs   RAM Storage      SSH Addr SSH Port    $/hr                                         Image Net up Net down     R
 0  398753    1810  running  1 x  Tesla K80   100.0  16.0  31.8       1  ssh4.vast.ai    38753  0.0252  tensorflow/tensorflow:1.14.0-gpu-py3-jupyter  423.8    495.8  98.4,
        ID Machine   Status  Num      Model Util. % vCPUs   RAM Storage      SSH Addr SSH Port    $/hr                                         Image Net up Net down     R
 0  398754    1810  running  1 x  Tesla K80   100.0  16.0  31.8       1  ssh5.vast.ai    38754  0.0252  tensorflow/tensorflow:1.14.0-gpu-py3-jupyter  423.8    495.8  98.4]

In [10]:
for inst in vast.get_instances():
    inst.start()

Starting instance 398753.
Starting instance 398754.


In [7]:
def filter_offers():
    return [offer for offer in vast.search_offers('dph', 'dph<.12 inet_down>200', instance_type='on-demand') 
        if offer['gpu_name'] and offer['id'] not in [397086, 395039, 396679]]
# print("%i offers."%len(offers))
#for offer in offers:
    #print(offer['id'],offer['dph_total'])

In [8]:
for offer in filter_offers():
    print(offer['id'], offer['dph_total'], offer['gpu_name'], offer['inet_down'], offer['inet_down'])

395017 0.025 Tesla K80 506.9 506.9
398574 0.05 P104-100 0.0 0.0
395016 0.05 Tesla K80 506.9 506.9
398629 0.08 GTX 1070 Ti 845.8 845.8
398570 0.085 GTX 1070 Ti 305.5 305.5
397896 0.085 GTX 1070 Ti 310.2 310.2
398423 0.09 GTX 1080 Ti 261.2 261.2
397085 0.09 GTX 1070 Ti 452.3 452.3
397078 0.09 GTX 1080 458.7 458.7
395482 0.09 GTX 1080 Ti 260.9 260.9
389091 0.09 GTX 1080 Ti 371.5 371.5
380556 0.09 GTX 1080 Ti 546.7 546.7
397083 0.095 GTX 1080 454.7 454.7
398576 0.1 P104-100 0.0 0.0
396977 0.1 GTX 1080 Ti 574.3 574.3
384637 0.1 GTX 1080 Ti 605.6 605.6
380573 0.1 GTX 1080 Ti 471.9 471.9
380541 0.1 GTX 1080 Ti 549.0 549.0
380534 0.1 GTX 1080 Ti 551.5 551.5
379152 0.1 GTX 1080 Ti 430.0 430.0
378011 0.1 GTX 1080 Ti 339.9 339.9
373301 0.11 GTX 1080 Ti 556.0 556.0
373300 0.11 GTX 1080 Ti 556.0 556.0


In [9]:
onstart = "#!/bin/sh\n"\
          "touch ~/.no_auto_tmux\n"\
          "apt-get install -y mongodb-server\n"\
          "service mongodb start\n"
resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
                          image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
mongo_instance_id = resp['new_contract']
time.sleep(30)
mongo_instance = vast.get_instance(mongo_instance_id)

onstart = "#!/bin/sh\n"\
          "touch ~/.no_auto_tmux\n"\
          "apt-get install -y mongodb-server\n"\
          "mkdir -p ~/.ssh\n"\
          "ssh-keygen  -P '' -f ~/.ssh/mongo\n"
resp=vast.create_instance(filter_offers()[0]['id'], disk=1, onstart_cmd=onstart,
                          image="tensorflow/tensorflow:1.14.0-gpu-py3-jupyter")
worker_instance_id = resp['new_contract']

time.sleep(30)
worker_instance = vast.get_instance(worker_instance_id)

https://vast.ai/api/v0/asks/395017/?api_key=e794a3273e5a3e74d545bbc368849ff263f726042cf92587df9519a6cda62e74 
 {"client_id": "me", "image": "tensorflow/tensorflow:1.14.0-gpu-py3-jupyter", "price": null, "disk": 1, "label": null, "onstart": "#!/bin/sh\napt-get install -y mongodb-server\nservice mongodb start\n", "runtype": "ssh", "python_utf8": false, "lang_utf8": false, "use_jupyter_lab": false, "jupyter_dir": null, "create_from": null, "force": false}
{"success": true, "new_contract": 398753}
https://vast.ai/api/v0/asks/395015/?api_key=e794a3273e5a3e74d545bbc368849ff263f726042cf92587df9519a6cda62e74 
 {"client_id": "me", "image": "tensorflow/tensorflow:1.14.0-gpu-py3-jupyter", "price": null, "disk": 1, "label": null, "onstart": "#!/bin/sh\napt-get install -y mongodb-server\nmkdir -p ~/.ssh\nssh-keygen  -P '' -f ~/.ssh/mongo\n", "runtype": "ssh", "python_utf8": false, "lang_utf8": false, "use_jupyter_lab": false, "jupyter_dir": null, "create_from": null, "force": false}
{"success": tru

In [24]:
mongo_instance.wait_until_running()

       ID Machine   Status  Num      Model Util. % vCPUs   RAM Storage      SSH Addr SSH Port    $/hr                                         Image Net up Net down     R
0  398753    1810  running  1 x  Tesla K80     0.0  16.0  31.8       1  ssh4.vast.ai    38753  0.0252  tensorflow/tensorflow:1.14.0-gpu-py3-jupyter  402.8    506.9  98.4

In [40]:
worker_instance.wait_until_running()

status is 'loading', waiting 60s...


       ID Machine   Status  Num        Model Util. % vCPUs   RAM Storage      SSH Addr SSH Port    $/hr                                         Image Net up Net down     R
0  398092     940  running  1 x  GTX 1070 Ti     0.0   4.0  32.0       1  ssh5.vast.ai    26322  0.0864  tensorflow/tensorflow:1.14.0-gpu-py3-jupyter  302.2    314.6  97.8

In [26]:
worker_machine = ParamikoMachine(worker_instance.ssh_host, user='root', port=worker_instance.ssh_port, 
                         keyfile=vast._get_ssh_key_file(), missing_host_policy=AutoAddPolicy)
mongo_machine = ParamikoMachine(mongo_instance.ssh_host, user='root', port=mongo_instance.ssh_port, 
                         keyfile=vast._get_ssh_key_file(), missing_host_policy=AutoAddPolicy)
status_resp=mongo_machine['service'](['mongodb','status'])
display(status_resp.split('\n')[-5:])

[' * Checking status of database mongodb', '   ...done.', '']

In [27]:
worker_machine.path('~/.ssh/mongo.pub').isfile()

True

In [28]:
pub_key=worker_machine['cat'](worker_machine.path('~/.ssh/mongo.pub'))
pub_key

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n'

In [ ]:
mongo_machine.session().run('echo "\n%s" >> %s'%(pub_key, mongo_machine.path('~/.ssh/authorized_keys')))

In [30]:
mongo_machine['cat'](mongo_machine.path('~/.ssh/authorized_keys'))

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost\nssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCwQNWg4ud+cP01AhsHEPohGD7/XRafkPiR0PpUrxMX6NOZUgOYuQzK9EN/667HhTnYJm6Yhez/gNGM8/GGzvuDoQQZazEwpnDtCQsgzS9wjBqmxT7pb2jeLuzZGGoXGMN2tY2dNw6TOJvc80QJKtU3hj4NEynTPi96uz888iClf6Gc8/H9rZYkZGF8jBgm5i8ONlbMWDHGlFryOtbuW2XZQZ+Li7ZP8Ad3ozk1pcZIW26IFcihqlxo8sEh+nHyY/Xsyt1bfjy6B7V3rDOBradwxnH5JGiSIZjobLRXrVJ4xxEk/GfJrhywdK3g8lTwVBUdasXJpI08m3P27sszlleB root@b9df8fc9c11d\n\n'

In [32]:
worker_instance.get_ssh_connection_command()

'ssh root@ssh5.vast.ai -p 38754 -i /home/sam/.ssh/vastai'

In [33]:
mongo_instance.get_ssh_connection_command()

'ssh root@ssh4.vast.ai -p 38753 -i /home/sam/.ssh/vastai'

In [81]:
"%s"%mongo_machine.path('~/.ssh/mongo.pub')

'/root/.ssh/mongo.pub'

In [13]:
mongo_instance = vast.get_instance(398753)

In [14]:
worker_instance = vast.get_instance(398754)

In [28]:
mongo_machine = mongo_instance.pb_remote()
mongo_machine

<ParamikoMachine paramiko://root@ssh4.vast.ai>

In [31]:
worker_machine = mongo_instance.pb_remote()
worker_machine

<ParamikoMachine paramiko://root@ssh4.vast.ai>

In [16]:
print(mongo_machine['pwd']())

/root



In [34]:
mongo_machine['cat']("/root/.ssh/authorized_keys")

'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDaKOIegXeknsbwKsyvY+sdgRFn71kg9yr64ktpV17ImXrggQ+LHmhqMhbv/1gNQhNKIRJz7FllaxxQZkeOnqcv/tLpom3NaZG7OMWkfRR7IK1Apv4n9J2sGUIHlBTyjdDmawiXlttzDjphcR0ZSZ58V+8Hr/S7L+pb1nkXmSY109LV6XjYjo1DPmngU+lnAFilErV3fSZFnmKDA95xQH+ZVN91nKO/jsUOLAIeIo4o5aSR/nv8KGB9GpKbVAueXPji/HrWijXuWSepEtbhtBq8cXTFqgoakhD/Cz4kmZIqwVOLRU+OZRq+xFsJK1UQBR34GRcQeda5AKQ7tDypK2Rj sam@localhost'

In [35]:
mongo_machine['cat']('onstart.sh')

'#!/bin/sh\napt-get install -y mongodb-server\nservice mongodb start\n'

In [36]:
mongo_instance.get_ssh_connection_command()

'ssh root@ssh4.vast.ai -p 38753 -i /home/sam/.ssh/vastai'

In [40]:
list(mongo_machine.pgrep(""))

[ProcInfo(1, 0, 'Ss+', '/bin/bash -c while [ ! -e /.launch ]; do sleep 1; done; bash /.launch'),
 ProcInfo(6, 0, 'S+', 'bash /.launch'),
 ProcInfo(16, 0, 'Ss', '/usr/sbin/sshd'),
 ProcInfo(21, 0, 'S+', 'ssh -i /etc/forward_key -N -o TCPKeepAlive=yes -o ServerAliveInterval=10 -o ServerAliveCountMax=2 ubuntu@ssh4.vast.ai -o StrictHostKeyChecking=no -o ConnectTimeout=10 -o RequestTTY=no -o ExitOnForwardFailure=yes -R \\*:38753:localhost:22'),
 ProcInfo(44, 103, 'Sl', '/usr/bin/mongod --config /etc/mongodb.conf'),
 ProcInfo(177, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(192, 0, 'Ss', '-bash'),
 ProcInfo(198, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(200, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(215, 0, 'Ss', '-bash'),
 ProcInfo(221, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(541, 0, 'Rs', '/bin/ps -e -o pid,uid,stat,args')]

In [41]:
list(worker_machine.pgrep(''))

[ProcInfo(1, 0, 'Ss+', '/bin/bash -c while [ ! -e /.launch ]; do sleep 1; done; bash /.launch'),
 ProcInfo(6, 0, 'S+', 'bash /.launch'),
 ProcInfo(16, 0, 'Ss', '/usr/sbin/sshd'),
 ProcInfo(21, 0, 'S+', 'ssh -i /etc/forward_key -N -o TCPKeepAlive=yes -o ServerAliveInterval=10 -o ServerAliveCountMax=2 ubuntu@ssh4.vast.ai -o StrictHostKeyChecking=no -o ConnectTimeout=10 -o RequestTTY=no -o ExitOnForwardFailure=yes -R \\*:38753:localhost:22'),
 ProcInfo(44, 103, 'Sl', '/usr/bin/mongod --config /etc/mongodb.conf'),
 ProcInfo(177, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(192, 0, 'Ss', '-bash'),
 ProcInfo(198, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(200, 0, 'Ss', 'sshd: root@notty'),
 ProcInfo(215, 0, 'Ss', '-bash'),
 ProcInfo(221, 0, 'Ss', '/usr/lib/openssh/sftp-server'),
 ProcInfo(542, 0, 'Rs', '/bin/ps -e -o pid,uid,stat,args')]

In [42]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.


In [150]:
vast.get_instances()

[398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter,
 398754: offline   $0.0252/hr ssh5.vast.ai:38754  423.8↑ 495.8↓  flops:8.736T
 	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
 	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter]

In [86]:
inst=vast.get_instances()[0]

In [147]:
inst

398753: offline   $0.0252/hr ssh4.vast.ai:38753  423.8↑ 495.8↓  flops:8.736T
	1X 11.2GB Tesla K80, 16.0/32X 15.7GB Xeon® Silver 4108 
	Successfully loaded tensorflow/tensorflow:1.14.0-gpu-py3-jupyter

In [149]:
vast.stop_all_instances()

Stopping instance 398753.
Stopping instance 398754.
